In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
#from sklearn.experimental import enable_hist_gradient_boosting
#from sklearn.ensemble import HistGradientBoostingClassifier
from joblib import dump, load
from sklearn.naive_bayes import GaussianNB 
from VisionUtils import *
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.multiclass import OneVsRestClassifier
import pandas as pd
from mtcnn.mtcnn import MTCNN
from pathlib import Path
from sklearn.utils import shuffle
import autosklearn.classification

ModuleNotFoundError: No module named 'autosklearn'

In [5]:
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install

'xargs' is not recognized as an internal or external command,
operable program or batch file.


In [72]:
def trans(feat1, feat2):
    f1 = [feat1[i1].reshape(-1,1) for i1 in range(len(feat1))]
    f2 = [feat2[i2].reshape(-1,1) for i2 in range(len(feat2))]
    cos_d = np.array([feat_distance_cosine_scalar(f1[i].T, f2[i]) for i in range(len(feat1))])
    cos_d = cos_d.reshape(-1,1)
    sqr_diff = np.power(np.abs(feat1- feat2), 2)
    rat = feat1/feat2
    data = np.hstack([cos_d, sqr_diff, rat])
    return data

In [90]:
train_aug_feat1 = np.load("features/train_aug_feat12.npy")
train_aug_feat2 = np.load("features/train_aug_feat22.npy")
train_aug_labels = np.load("features/train_aug_labels2.npy")

train_parallel_feat1 = np.load("features/train_parallel_feat11.npy")
train_parallel_feat2 = np.load("features/train_parallel_feat21.npy")
train_parallel_labels = np.load("features/train_parallel_labels.npy")

train_spotlight_feat1 = np.load("features/train_spotlight_feat11.npy")
train_spotlight_feat2 = np.load("features/train_spotlight_feat21.npy")
train_spotlight_labels = np.load("features/train_spotlight_labels.npy")

train_o_feat1 = np.load("features/train_feat1.npy")
train_o_feat2 = np.load("features/train_feat2.npy")
train_labels = np.load("features/train_labels.npy")

test_feat1 = np.load("features/test_feat1.npy")
test_feat2 = np.load("features/test_feat2.npy")
test_labels = np.load("features/test_labels.npy")

In [91]:
labels =  list(train_aug_labels) + list(train_spotlight_labels) +list(train_parallel_labels) 

labels = labels + list(train_labels) + list(train_labels) + list(train_labels) 
labels = labels + list(test_labels)

In [92]:
train_feat1 = np.vstack([train_aug_feat1, train_spotlight_feat1, train_parallel_feat1, 
                         train_o_feat1, train_o_feat1, train_o_feat1])
train_feat2 = np.vstack([train_aug_feat2, train_spotlight_feat2, train_parallel_feat2, 
                         train_o_feat2, train_o_feat2, train_o_feat2])

In [93]:
train_feat2.shape

(246912, 128)

In [94]:
train_feat1.shape

(246912, 128)

In [95]:
xtrain = trans(train_feat1, train_feat2)
xtest = trans(test_feat1, test_feat2)

In [96]:
tr_len = len(xtrain)
tst_len = len(xtest)

In [97]:
data = np.vstack([xtrain, xtest])


In [98]:
data.shape
#labels.shape

(259801, 257)

In [99]:
len(labels)

259801

In [100]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)
f_selector = SelectKBest(chi2, k = int(0.8*(len(data.T))))
data_ = f_selector.fit_transform(scaled_data, labels )

data_df = pd.DataFrame(data_)
image_paths_csv = pd.read_csv("features/image_paths.csv", index_col = False)

all_data_df = pd.concat([image_paths_csv,data_df], axis = 1)

len(all_data_df)

xtrain, xtest, ytrain, ytest = train_test_split(all_data_df, labels, shuffle = True, random_state = 42)

paths_dict = {
    "train_paths1": xtrain["path1"],
    "train_paths2": xtrain["path2"],
    "test_paths1": xtest["path1"],
    "test_paths2": xtest["path2"]
}
xtrain1 = xtrain.drop(["path1", "path2", "Unnamed: 0"], axis = 1).values
xtest1 = xtest.drop(["path1", "path2", "Unnamed: 0"], axis = 1).values

In [101]:
xtrain, xtest = data_[:tr_len], data_[-tst_len:]
ytrain, ytest = labels[:tr_len], labels[-tst_len:]
xtrain, ytrain = shuffle(xtrain, ytrain)

In [102]:
xtrain.shape

(246912, 205)

In [103]:
xtest.shape

(12889, 205)

Gradient Boosting Classifier

In [104]:
from sklearn.ensemble import BaggingClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV


params_grid = {
    "loss" : ["deviance", "exponential"],
    "n_estimators": [10, 50,100, 150, 200],
              }
gboost = GradientBoostingClassifier()
gs = GridSearchCV(gboost, param_grid = params_grid, cv = 5 )
gs.fit(xtrain, ytrain)
report(gs.cv_results_)
gboost = gs.best_estimator_

In [105]:
gboost = GradientBoostingClassifier()

In [106]:
params_grid = {
    "loss" : ["deviance", "exponential"],
    "n_estimators": [10, 50,100, 150, 200],}

gs = GridSearchCV(gboost, param_grid = params_grid, cv = 5 )

In [ ]:
gs.fit(xtrain, ytrain)

In [ ]:
gboost = gs.best_estimator_


In [68]:
gboost.fit( xtrain, ytrain)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [69]:
pred = gboost.predict(xtest)

In [70]:
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score 
acc = accuracy_score(ytest,pred)
conf = confusion_matrix(ytest, pred)
rs = recall_score(ytest, pred)
ps = precision_score(ytest, pred)
f1 = f1_score(ytest, pred)
print("-The Accuracy of the the cllasifier:",acc)
print("-The recall Score: ", rs)
print("-The precision score: ", ps)
print("-The F1_score: ", f1)
print("-The confusion matrix:" )
print(conf)

-The Accuracy of the the cllasifier: 0.8836216929164403
-The recall Score:  0.8462128181539406
-The precision score:  0.9173869680851063
-The F1_score:  0.8803636943691179
-The confusion matrix:
[[5870  497]
 [1003 5519]]


In [31]:
false_positive = []
false_positive_pred = []

false_negative = []
false_negative_pred = []

for i in range(len(pred)):
    ### False Positive check
    if ((pred[i]==1) & (ytest[i] ==0)):
        false_positive.append(i)
        false_positive_pred.append(pred[i])
    ### False Negative check
    elif ((pred[i] == 0) &(ytest[i]==1)):
        false_negative.append(i)
        false_negative_pred.append(pred[i])


In [32]:
print(len(false_positive))
print(len(false_negative))

285
452


False Positves

In [33]:
false_image_lst1 = list(np.array(paths_dict["test_paths1"])[false_positive])
false_image_lst2 = list(np.array(paths_dict["test_paths2"])[false_positive])

NameError: name 'paths_dict' is not defined

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt 

def showfalse(idx, i):
    
    try:
        imgs = [ plt.imread("data/images/" + false_image_lst1[idx]), 
               plt.imread("data/images/" + false_image_lst2[idx])]
        pathls = ["#: {} data/images/".format(i) + false_image_lst1[idx], 
                  "#: {} data/images/".format(i) + false_image_lst2[idx]]
        for i in range(len(imgs)):
            bb = detect_faces(imgs[i], m)
            ax = show_img(imgs[i])
            
            for b in bb:
                write_txt(ax, (b[0], b[1]),pathls[i], 14)
                draw_bb(ax, b)
            write_txt(ax, (0, 0),"{}".format(i), 20)
        #print(50*"#")
    except FileNotFoundError:
        print(i)

In [18]:

os.path.exists("data/images/" + false_image_lst1[1]) 
os.path.exists("data/images/" + false_image_lst1[1])

True

In [46]:
dump(gboost, "demo/PipelineParts/GboostModel.joblib")
dump(scaler, "demo/PipelineParts/scaler.joblib")
dump(f_selector, "demo/PipelineParts/feature_selector.joblib")

['demo/PipelineParts/feature_selector.joblib']